In [1]:
vm_file = '/hdd/jbinin/alibaba2018_data_extraction/data/hole'
import os
import pandas as pd
def read_iterator(filepath,readNumPrecent=0.125):
    cpulist = []
    memlist = []
    files = os.listdir(filepath)
    n = int(69119*readNumPrecent)
    for idx,file in enumerate(files):
        if idx > 3:
            break
        filename = os.path.join(filepath, file)
        with open(filename) as f:
            cpus = pd.read_csv(f,iterator=True,header=None)
            cpus = cpus.get_chunk(n)
            cpu = cpus[0].values.squeeze().tolist()
            mem = cpus[1].values.squeeze().tolist()
            cpulist.append(cpu[:int(len(cpu)*readNumPrecent)])
            memlist.append(mem[:int(len(mem)*readNumPrecent)])
        print(idx,',',filename,',',n)
    return cpulist,memlist
cpus = read_iterator(vm_file)


0 , /hdd/jbinin/alibaba2018_data_extraction/data/hole/instanceid_70769.csv , 8639
1 , /hdd/jbinin/alibaba2018_data_extraction/data/hole/instanceid_68923.csv , 8639
2 , /hdd/jbinin/alibaba2018_data_extraction/data/hole/instanceid_55527.csv , 8639
3 , /hdd/jbinin/alibaba2018_data_extraction/data/hole/instanceid_39415.csv , 8639


In [12]:
print( len(cpus[0][0][100:111]))

1079


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

def split_sequence(sequence, m_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(sequence)):

        end_element_index = i + n_steps_in
        out_end_index = end_element_index + n_steps_out
        if out_end_index > len(sequence): 
            break
        
        sequence_x, sequence_y = sequence[i:end_element_index], sequence[end_element_index:out_end_index]
        X.append(sequence_x)
        y.append(sequence_y)

    return np.array(X), np.array(y)

def vector_output_model(n_steps_in, n_steps_out, X, y, epochs_num):
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim=n_steps_in))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    
    model.fit(X, y, epochs=2000, verbose=0)
    return model


epochs_num = 2000
n_steps_in, n_steps_out = 3, 2

raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
raw_seq = cpus[0][0][100:111]
print(raw_seq)
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

print(X.shape, y.shape)
for i in range(len(X)):
    print(X[i], y[i])

model = vector_output_model(n_steps_in, n_steps_out, X, y, epochs_num)

x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in))
yhat = model.predict(x_input, verbose=0)
print(yhat)


[0.0, 0.0, 0.08, 0.08, 0.0, 0.0, 0.08, 0.0, 0.08, 0.08, 0.0]
(7, 3) (7, 2)
[0.   0.   0.08] [0.08 0.  ]
[0.   0.08 0.08] [0. 0.]
[0.08 0.08 0.  ] [0.   0.08]
[0.08 0.   0.  ] [0.08 0.  ]
[0.   0.   0.08] [0.   0.08]
[0.   0.08 0.  ] [0.08 0.08]
[0.08 0.   0.08] [0.08 0.  ]
[[-62.489918 -25.795572]]
